Let's start lookinf for classic citations at the EuropePMC database.

Once more, we will use naive definitions of citation classics. 

Classics are all cited more than 150 times. 

Big classics: more than 500 citations

Medium classics: at least 300 citations, less than 500

Small classics: at least 150 citations, less than 300


In [13]:
import json
from pandas.io.json import json_normalize


import requests
from bs4 import BeautifulSoup

pmc_query = "https://www.ebi.ac.uk/europepmc/webservices/rest/search?query=%22single-cell%20RNA-seq%22&resultType=lite&cursorMark=*&pageSize=100&sort=CITED%20desc&format=json"
page = requests.get(pmc_query)

data = json.loads(page.text)
df = json_normalize(data['resultList']['result'])


single_cell_papers = df[['pmid', 'title', 'authorString', 'journalTitle', 'citedByCount', 'pubYear']]



In [19]:
def get_first_author(string):
    result = string.split(',')[0]
    return(result)
    
first_authors = single_cell_papers['authorString'].apply(get_first_author)
single_cell_papers['firstAuthor'] = first_authors
single_cell_papers = single_cell_papers[single_cell_papers['citedByCount']>=150]

single_cell_papers

,pmid,title,authorString,journalTitle,citedByCount,pubYear,firstAuthor
0,26000488,Highly Parallel Genome-wide Expression Profili...,"Macosko EZ, Basu A, Satija R, Nemesh J, Shekha...",Cell,1092,2015,Macosko EZ
1,24925914,Single-cell RNA-seq highlights intratumoral he...,"Patel AP, Tirosh I, Trombetta JJ, Shalek AK, G...",Science,938,2014,Patel AP
2,25700174,Brain structure. Cell types in the mouse corte...,"Zeisel A, Muñoz-Manchado AB, Codeluppi S, Lönn...",Science,726,2015,Zeisel A
3,26000487,Droplet barcoding for single-cell transcriptom...,"Klein AM, Mazutis L, Akartuna I, Tallapragada ...",Cell,582,2015,Klein AM
4,24658644,The dynamics and regulators of cell fate decis...,"Trapnell C, Cacchiarelli D, Grimsby J, Pokhare...",Nat Biotechnol,569,2014,Trapnell C
...,...,...,...,...,...,...,...
59,27984732,Perturb-Seq: Dissecting Molecular Circuits wit...,"Dixit A, Parnas O, Li B, Chen J, Fulco CP, Jer...",Cell,158,2016,Dixit A
60,28212749,Comparative Analysis of Single-Cell RNA Sequen...,"Ziegenhain C, Vieth B, Parekh S, Reinius B, Gu...",Mol Cell,157,2017,Ziegenhain C
61,26431181,Adult Mammalian Neural Stem Cells and Neurogen...,"Bond AM, Ming GL, Song H.",Cell Stem Cell,151,2015,Bond AM
62,25471879,Deconstructing transcriptional heterogeneity i...,"Kumar RM, Cahan P, Shalek AK, Satija R, DaleyK...",Nature,151,2014,Kumar RM


Cool, now we have our database of interest: all articles with more than 150 citations in EuropePMC about "single-cell RNA sequencing". 

The goal now is to turn this database into a markdown, user-friendly list. 

We will start by creating a base markdown file


In [70]:
from mdutils.mdutils import MdUtils
mdFile = MdUtils(file_name='EuropePCM_classics', title='Classic citations from the Europe PMC database')

mdFile.new_header(level=1, title='Overview')
mdFile.new_paragraph("Let's start lookinf for classic citations at the EuropePMC database.Once more, we will use naive definitions of citation classics.") 


mdFile.new_line("*Classics are all cited more than 150 times.")
mdFile.new_line("*Big classics: more than 500 citations")
mdFile.new_line("*Medium classics: at least 300 citations, less than 500")
mdFile.new_line("*Small classics: at least 150 citations, less than 300\n")




"\n# Overview\n\n\nLet's start lookinf for classic citations at the EuropePMC database.Once more, we will use naive definitions of citation classics.  \n*Classics are all cited more than 150 times.  \n*Big classics: more than 500 citations  \n*Medium classics: at least 300 citations, less than 500  \n*Small classics: at least 150 citations, less than 300\n"

In [71]:
def get_years(pd):
    years = list(set(pd['pubYear']))
    years.sort(reverse = True)
    return(years)
    
    
def get_new_entry(row):
    result = ("* " + row['firstAuthor'] + " et al, " + "[" + row['title'] + "]" + 
         "(" + "https://europepmc.org/article/MED/" + row['pmid'] + ")")
    return(result)
    
    
def add_section_to_md(pd, mdFile):
    pd = pd.sort_values('firstAuthor')
    years = get_years(pd)

    for year in years:
        mdFile.new_header(3, year)
        df = pd[pd["pubYear"] == year]
        for index,row in df.iterrows():
           mdFile.new_line(get_new_entry(row))
        
        mdFile.new_line()
    
    return(mdFile)
    

In [72]:

mdFile.new_header(1, "Big classics")
big_classics = single_cell_papers[single_cell_papers['citedByCount'] >= 500]
mdFile = add_section_to_md(big_classics, mdFile)

mdFile.new_header(1, "Medium classics")
mid_classics = single_cell_papers[(single_cell_papers['citedByCount'] >= 300)
                                  & (single_cell_papers['citedByCount'] < 500)]
mdFile = add_section_to_md(mid_classics, mdFile)


mdFile.new_header(1, "Small classics")
smol_classics = single_cell_papers[(single_cell_papers['citedByCount'] >= 150)
                                  & (single_cell_papers['citedByCount'] < 300)]
mdFile = add_section_to_md(smol_classics, mdFile)

    

In [73]:
mdFile.create_md_file()
